In [20]:
!mkdir mmseg
!mkdir mmseg/labels
!mkdir mmseg/images
!mkdir mmseg/splits

In [32]:
from pycocotools.coco import COCO
import numpy as np
import os
import glob
import shutil

In [34]:
for path in glob.glob('batch_01_vt/*'):
    shutil.copy(path,'mmseg/images/'+path.replace('/','_'))

for path in glob.glob('batch_02_vt/*'):
    shutil.copy(path,'mmseg/images/'+path.replace('/','_'))

for path in glob.glob('batch_03/*'):
    shutil.copy(path,'mmseg/images/'+path.replace('/','_'))

In [22]:
val_dir = '../data/val.json'
train_dir = '../data/train.json'

val_coco = COCO(val_dir)
train_coco = COCO(train_dir)

[1]
{'license': 0, 'url': None, 'file_name': 'batch_01_vt/0005.jpg', 'height': 512, 'width': 512, 'date_captured': None, 'id': 1}


In [23]:
def make_txt(coco):
    for idx in coco.getImgIds():        # cocodata의 모든 image idx list
        image_id = coco.getImgIds(imgIds=idx)
        image_infos = coco.loadImgs(image_id)[0]            # 이미지 file_name, height, width를 담고있는 dict

        ann_ids = coco.getAnnIds(imgIds=image_infos['id'])  # 이미지 idx가 일치하는 ann id list
        anns = coco.loadAnns(ann_ids)

        masks = np.zeros((image_infos["height"], image_infos["width"]))
        for i in range(len(anns)):
            masks[coco.annToMask(anns[i]) == 1] = anns[i]['category_id']
        masks = masks.astype(np.int8).astype(np.str)

        with open(os.path.join('../data/mmseg/labels',image_infos['file_name'].replace('/','_').replace('.jpg','.txt')), 'w') as f:
            for mask in masks.tolist():
                f.write(' '.join(mask) + '\n')

In [24]:
make_txt(val_coco)
make_txt(train_coco)

In [25]:
cat_ids = val_coco.getCatIds()
cats = val_coco.loadCats(cat_ids)

General trash
Paper
Paper pack
Metal
Glass
Plastic
Styrofoam
Plastic bag
Battery
Clothing


In [26]:
import mmcv
import os.path as osp
import numpy as np
from PIL import Image

# convert dataset annotation to semantic segmentation map
data_root = 'mmseg'
img_dir = 'images'
ann_dir = 'labels'

# define class and plaette for better visualization
classes = []
for c in cats:
    classes.append(c['name'])
palette = [[128, 128, 128], [129, 127, 38], [120, 69, 125], [53, 125, 34], 
           [0, 11, 123], [118, 20, 12], [122, 81, 25], [241, 134, 51],[0, 241, 0],[0, 241, 0]]
for file in mmcv.scandir(osp.join(data_root, ann_dir), suffix='.txt'):
  seg_map = np.loadtxt(osp.join(data_root, ann_dir, file)).astype(np.uint8)
  seg_img = Image.fromarray(seg_map).convert('P')
  seg_img.putpalette(np.array(palette, dtype=np.uint8))
  seg_img.save(osp.join(data_root, ann_dir, file.replace('.txt','.png')))

In [29]:
with open('mmseg/splits/train.txt','w') as f:
    for idx in train_coco.getImgIds():
        image_id = train_coco.getImgIds(imgIds=idx)
        image_infos = train_coco.loadImgs(image_id)[0]
        f.write(image_infos['file_name'].replace('/','_').replace('.jpg','') + '\n')

In [30]:
with open('mmseg/splits/val.txt','w') as f:
    for idx in val_coco.getImgIds():
        image_id = val_coco.getImgIds(imgIds=idx)
        image_infos = val_coco.loadImgs(image_id)[0]    
        f.write(image_infos['file_name'].replace('/','_').replace('.jpg','') + '\n')